In [1]:
import os
import pandas as pd
import common
from shutil import copyfile

In [2]:
metadata = common.load_metadata_dataframe('state.json')

In [6]:
FROM_PREFIX = '/Users/fcuevas/Documents/imgs/{}'
TO_PREFIX = "./imgs/{}"

def copy_img(row):
    from_dir = FROM_PREFIX.format(row["image"])
    to_dir = TO_PREFIX.format(row["image"])
    
    os.makedirs(os.path.dirname(to_dir), exist_ok=True)
    copyfile(from_dir, to_dir)

metadata.apply(copy_img, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
2786    None
2787    None
2788    None
2789    None
2790    None
Length: 2791, dtype: object

In [5]:
metadata[metadata["useful"] == "yes"].shape

(1330, 3)